In [1]:
%gui osx
import py5
import numpy as np
import random
import copy
import time
import math

In [2]:
class mPoint:
    '''ez a pontok (csúcsok) osztálya'''
    def __init__(self,inx, iny, index=-1, szomszedok={}):
        self.x =inx
        self.y =iny
        self.index=index
        self.szomszedok=szomszedok
      
    
    def __sub__(self, other):
        '''kivonja két csúcs koordinátáit egymásból'''
        return mPoint(self.x-other.x,self.y-other.y)
    
    def __str__(self):
        '''így írunk ki egy csúcsot'''
        return str('('+str(self.index)+','+str(self.x)+','+str(self.y)+','+str(self.szomszedok)+')')
    
    def mag(self):
        '''a csúcs mint vektor hossza'''
        return np.sqrt(self.magSq())
  
    def magSq(self):
        '''a csúcs mint vektor hossznégyzete'''
        return self.x*self.x+self.y*self.y;
    
    def display(self):      
        '''csak a pontot magát rajzolja ki'''
        py5.stroke_weight(20)
        py5.point(self.x, self.y)
    
    def display_elestol(self):   
        '''kirajzolja a pontot és a belőle kiinduló éleket is'''
        py5.stroke_weight(20)
        py5.point(self.x, self.y)
        py5.stroke_weight(4)
        for csucs in self.szomszedok:
            py5.line(self.x,self.y,graf_csucsai.plist[csucs].x,graf_csucsai.plist[csucs].y)


In [3]:
def pontok_szoge(A,B,C): 
    '''az inputja három mPoint és kiszámolja a BAC szöget'''
    oszto1=B.x-A.x
    oszto2=C.x-A.x
    if oszto1==0:
        oszto1=10**(-10)
    if oszto2==0:
        oszto2=10**(-10)
    return abs(math.atan((B.y-A.y)/oszto1)-math.atan((C.y-A.y)/oszto2))

In [4]:
def orientation(p, q, r): 
    '''az inputja három mPoint megmondja a (p,q,r) irányítását'''
    
    val = (float(q.y - p.y) * (r.x - q.x)) - (float(q.x - p.x) * (r.y - q.y))
    if (val > 0):       #negatív irányítás
        return 1
    elif (val < 0):     #pozitív irányítás
        return 2
    else:              #kollineárisak
        return 0
  

In [5]:
def doIntersect_eros(p1,q1,p2,q2,szog=0.08): 
    '''az inputja négy mPoint és megmondja, hogy a p1q1 és a p2q2 szakaszok metszenek-e vagy a szognel kisebb szöget zárnak-e be'''
    
    if p1==p2:
        if pontok_szoge(p1,q1,q2)<szog:
            return True
        else:
            return False
    if p1==q2:
        if pontok_szoge(p1,q1,p2)<szog:
            return True
        else:
            return False
    if q1==p2:
        if pontok_szoge(p2,p1,q2)<szog:
            return True
        else:
            return False
    if q1==q2:
        if pontok_szoge(q1,p1,p2)<szog:
            return True
        else:
            return False  
    
    o1 = orientation(p1, q1, p2)
    o2 = orientation(p1, q1, q2)
    o3 = orientation(p2, q2, p1)
    o4 = orientation(p2, q2, q1)
  
    if ((o1 != o2) and (o3 != o4)):
        return True
    else:
        return False
    
def doIntersect(p1,q1,p2,q2): 
    '''az inputja négy mPoint és megmondja, hogy a p1q1 és a p2q2 szakaszok metszenek-e'''
    
    if (p1==p2) or (p1==q2) or (q1==p2) or (q1==q2):
        return False
    
    o1 = orientation(p1, q1, p2)
    o2 = orientation(p1, q1, q2)
    o3 = orientation(p2, q2, p1)
    o4 = orientation(p2, q2, q1)
  
    if ((o1 != o2) and (o3 != o4)):
        return True
    else:
        return False

In [6]:
class edge:
    '''ez a gráf éleinek osztálya'''
    global graf_elei
    def __init__(self, inStart,inEnd):
        self.s =inStart
        self.t =inEnd
        self.metszo=False #ez a tulajdonság mondja meg, hogy van-e olyan éle a gráfnak, amit ez metsz (ez alapján színezzük ki)
        
    def __str__(self):
        '''csak azt íratjuk ki, hogy melyik két indexű csúcs között megy az él'''
        return '['+str(self.s.index)+','+str(self.t.index)+']'
       
    def display(self):
        '''attól függően, hogy metsz-e másik élet szürkével vagy narancssárgával írjuk ki'''
        py5.stroke_weight(4)
        if self.metszo==True:
            py5.stroke(204, 102, 0)
        else:
            py5.stroke(153)
            
        py5.line(self.s.x,self.s.y,self.t.x,self.t.y)
        
    def metsz_e_egy_konkretat(self,masikel):      
        '''edldönti, hogy az élünk metszi-e a másik megadott élet'''
        return doIntersect(self.s,self.t,masikel.s,masikel.t)
    
    def metsz_e_egy_konkretat_eros(self,masikel):      
        '''edldönti, hogy az élünk metszi-e a másik megadott élet vagy kis szöget zár be vele'''
        return doIntersect_eros(self.s,self.t,masikel.s,masikel.t)
   
        
    def metsz_e_valamit(self):                     
        '''van-e olyan él a gráfban, amit ez metsz, ha van, akkor a self.metszo-t átállítjuk, mert ez alapján rajzoljuk ki'''
        ideig=False
        for el in graf_elei.plist:
            if (self.metsz_e_egy_konkretat(el)) and (self != el):
                ideig=True
        self.metszo=ideig
        return ideig
    
    def metsz_e_valamit_eros(self):                     
        '''van-e olyan él a gráfban, amit ez metsz vag kis szöget zár be vele'''
        ideig=False
        for el in graf_elei.plist:
            if (self.metsz_e_egy_konkretat_eros(el)) and (self != el):
                ideig=True
        return ideig

In [7]:
class edge_set:
    '''ez a gráf összes élének az osztálya'''
    
    def __init__(self, plist=[]):
        self.plist=plist
        
    def display(self):
        for x in self.plist:
            x.display()
            
    def __str__(self):
        szoveg=""
        for el in self.plist:
            szoveg+=str(el)
        return szoveg
    
    def van_e_metszes(self):      
        '''eldönti, hogy a gráf élei között vannak-e metszőek'''
        ideig=False
        for x in self.plist:
            if x.metsz_e_valamit():
                ideig=True
        return ideig
    
    def van_e_metszes_eros(self):      
        '''eldönti, hogy a gráf élei között vannak-e metszőek vagy kis szöget bezáróak'''
        ideig=False
        for x in self.plist:
            if x.metsz_e_valamit_eros():
                ideig=True
        return ideig
    
    def el_benne_van_e(self, csucs1,csucs2): 
        '''az adott él szerepel-e a gráfban'''
        for el in self.plist:
            if ((el.s.index,el.t.index)==(csucs1,csucs2)) or ((el.s.index,el.t.index)==(csucs2,csucs1)):
                return True
        return False

In [8]:
def dist( a, b):
    return (a-b).mag();           
            
def mouse_point():
    return mPoint(py5.mouse_x-py5.width/2,py5.mouse_y-py5.height/2)

In [9]:
class dinamic_pointset:
    '''ez a gráf összes csúcsának az osztály'''
    def __init__(self, plist=[]):
        self.plist=plist
        self.active=False
        self.act_point=mPoint(0,0,0,[])
        self.log=False         
       
    def display(self):
        for x in self.plist:
            x.display()
    
    def select(self):
        if self.log:
            print('select')
        
        ref_point=mPoint(py5.mouse_x-py5.width/2,py5.mouse_y-py5.height/2)
        ref_point.display()
        dist_bound=30
        min_dist=dist_bound
        min_point=mPoint(0,0)
        for p in self.plist:
            if dist(ref_point,p)<min_dist:
                min_dist=dist(ref_point,p)
                min_point=p
        
        if min_dist<dist_bound:
            if self.log:
                print('succes select')
        
            self.active=True
            self.act_point=min_point
        else:
            if self.log:
                print('not succes select')
     #   self.plist.append(ref_point)
        
    def update(self):
        '''a kijelölt pont koordinátáit az egérére változttja'''
        if self.active:
            self.act_point.x=py5.mouse_x-py5.width/2
            self.act_point.y=py5.mouse_y-py5.height/2
    
    def relase(self):
        if self.log:
            print('re')
        
        self.active=False
   
    def do_something(self):
        if self.active:
            self.relase()
        else:
            self.select()



In [10]:
def random_adjacencia_matrix(num_of_points):
    matrix=np.random.randint(2,size=(num_of_points,num_of_points))
    for i in range(num_of_points):
        for j in range(num_of_points):
            if i==j:
                matrix[i][j]=0
            elif i<j:
                matrix[i][j]=matrix[j][i]
    return matrix

In [11]:
def create_graph(num_of_points):
    global graf_csucsai
    global graf_elei
    points=np.random.rand(num_of_points,2)*700-350
    adjacencia_matrix=random_adjacencia_matrix(num_of_points)
    ideig_ponthalmaz=[]
    i=0
        
    for x,y in points:
        random_csucshalmaz=set(j for j in range(num_of_points) if adjacencia_matrix[i][j]==1)
        ideig_ponthalmaz.append(mPoint(x,y,i,random_csucshalmaz))
        i+=1
    graf_csucsai=dinamic_pointset(ideig_ponthalmaz)
        
    ideiglenes_elhalmaz=[]
    for i in range(num_of_points):
        for j in range(num_of_points):
            if adjacencia_matrix[i][j]==1:
                ideiglenes_elhalmaz.append(edge(graf_csucsai.plist[i],graf_csucsai.plist[j]))
            
    graf_elei=edge_set(ideiglenes_elhalmaz)

In [12]:
def random_ponthalmaz(num_of_points):
    '''random koordinátákat generál a pontjainknak, amik nincsenek túl közel egymáshoz'''
    points=[]
    i=0
    while (len(points)<num_of_points):
        i+=1
        x_koord=np.random.rand()*(hossz-240)-hossz//2+230
        y_koord=np.random.rand()*(szel-20)-szel//2+10
        new_point=(x_koord, y_koord)
        ideig_bool=True
        for csucs in points:
            if (new_point[0]-csucs[0])**2+(new_point[1]-csucs[1])**2 < 20000: #ha túl közel van az új pont, nem veszi bele
                ideig_bool=False
        if ideig_bool:
            points.append(new_point)
    return points

In [13]:
def create_graph2(num_of_points): 
    '''generál egy random gráfot, ezt belerakja a a graf_elei és graf_csucsai classokba'''
    
    global graf_csucsai
    global graf_elei
    global eredeti_csucsok
    
    points=random_ponthalmaz(num_of_points)
    ideig_ponthalmaz=[]
        
    for k in range(num_of_points): #definialjuk a graf_csucsai dinamic_pointset-et a fent generált random koordinátákkal
        ideig_ponthalmaz.append(mPoint(points[k][0],points[k][1],k,set({})))
    graf_csucsai=dinamic_pointset(ideig_ponthalmaz)
    eredeti_csucsok=copy.deepcopy(graf_csucsai)
        
    ideiglenes_elhalmaz=[]
    ideiglenes_elhalmaz_regi=[]
    for _ in range(num_of_points**(nehezseg+1)):  #itt jön be a nehézség, hogy hányszor próbál új élet behúzni
        csucs1=random.randint(0,num_of_points-1)
        csucs2=random.randint(0,num_of_points-1)
        graf_elei=edge_set(ideiglenes_elhalmaz)
        if (csucs1==csucs2) or graf_elei.el_benne_van_e(csucs1,csucs2): #olyan élet veszünk, ami még nincs a gráfban
            continue
        else:
            ideiglenes_elhalmaz.append(edge(graf_csucsai.plist[csucs1],graf_csucsai.plist[csucs2]))
            graf_elei=edge_set(ideiglenes_elhalmaz)
            if graf_elei.van_e_metszes_eros():   #ha van metszés az új éllel, akkor visszatérünk a régi listához
                ideiglenes_elhalmaz=ideiglenes_elhalmaz_regi.copy()
            else:                            #ha nincs metszés, akkor a régit frissítjük
                ideiglenes_elhalmaz_regi=ideiglenes_elhalmaz.copy()
            graf_elei=edge_set(ideiglenes_elhalmaz)

            
    for el in graf_elei.plist:   #frissítjük a csúcsoknál is a szomszédjaikat
        for i in range(len(graf_csucsai.plist)):
            if el.s.index==i:
                graf_csucsai.plist[i].szomszedok.add(el.t.index)
                eredeti_csucsok.plist[i].szomszedok.add(el.t.index)
            elif el.t.index==i:
                graf_csucsai.plist[i].szomszedok.add(el.s.index)  
                eredeti_csucsok.plist[i].szomszedok.add(el.s.index)
                
    shuffle_graph()

In [14]:
def shuffle_graph():
    '''random módon újraválasztja a csúcsok koordinátáit'''
    global hossz
    global szel
    new_points_x=np.random.rand(num_of_points)*(hossz-240)-hossz//2+230
    new_points_y=np.random.rand(num_of_points)*(szel-20)-szel//2+10
    i=0
    for csucs in graf_csucsai.plist:
        csucs.x=new_points_x[i]
        csucs.y=new_points_y[i]
        i+=1
    

In [15]:
def setup():
    global hossz
    global szel
    py5.size(hossz, szel)
    py5.rect_mode(py5.CENTER)

def mouse_released(e):
    global graf_csucsai
    
    modifiers = e.get_modifiers()
    msgs = []
    graf_csucsai.do_something()

        

In [16]:
graf_csucsai=dinamic_pointset()
graf_elei=edge_set()
arany = 0
giveup_box_pressed = False
input_box_pressed = False
rectangles = [(-220,0,200,80),(0,0,200,80),(220,0,200,80)]
options = [10,12,15]
kesz = False

def draw(): #ez most a mindenes draw
    # adatok
    global hossz
    global szel
    global num_of_points
    global graf_csucsai
    global arany
    global giveup_box_pressed #ez valahogy tenyleg pressed kellene legyen
    global input_box_pressed
    global rectangles
    global options
    global kesz
    
    

    if py5.is_mouse_pressed:       #
        for i in range(len(rectangles)):
            if rectangles[i][0]-100 <= py5.mouse_x-py5.width/2 <= rectangles[i][0]+100 and rectangles[i][1]-40 <= py5.mouse_y-py5.height/2 <= rectangles[i][1]+40 and not input_box_pressed:
                num_of_points = options[i]
                create_graph2(num_of_points)
                input_box_pressed = True
                
        if -hossz//2+20 <= py5.mouse_x-py5.width/2 <= -hossz//2+220 and -szel//2+10 <= py5.mouse_y-py5.height/2 <= -szel//2+90 and input_box_pressed: #itt allitodik at igazra, ha odaviszed az egeret
            giveup_box_pressed = True
    
    if input_box_pressed:
        if not giveup_box_pressed: #ha nincs a gombon, akkor csak a szokasos
            mmouse_x=py5.mouse_x-py5.width/2
            mmouse_y=py5.mouse_y-py5.height/2
            graf_csucsai.update()
            graf_elei.van_e_metszes()
            if not graf_elei.van_e_metszes() and py5.is_mouse_pressed:
                kesz = True
        else: #ha ott van a gombon, akkor helyremennek a csucsok
            time.sleep(0.05)
            if arany<100:
                arany=arany+1
                for i in range(len(graf_csucsai.plist)): #itt mozognak a helyukre a csucsok
                    graf_csucsai.plist[i].x=(1-(arany/100))*graf_csucsai.plist[i].x+(arany/100)*eredeti_csucsok.plist[i].x
                    graf_csucsai.plist[i].y=(1-(arany/100))*graf_csucsai.plist[i].y+(arany/100)*eredeti_csucsok.plist[i].y
                graf_elei.van_e_metszes()
    
    #stílus
    py5.background(255)
    py5.translate(py5.width/2, py5.height/2);
    
    if not input_box_pressed:
        py5.text("Mozgasd át a gráf csúcsait úgy, hogy az élek ne metsszék egymást!",0,-200)
        py5.text("Hány csúcsa legyen a gráfnak?", 0, -100)
        for i in range(len(rectangles)):
            py5.stroke(0) # Change the stroke color to black
            py5.fill(204, 102, 0) # Change the fill color to light gray
            py5.rect(*rectangles[i])
            py5.text_size(24)
            py5.fill(0) # Change the fill color to black
            py5.text_align(py5.CENTER, py5.CENTER)
            py5.text(options[i], rectangles[i][0], rectangles[i][1])
    else:
        if not giveup_box_pressed and kesz:
            py5.stroke(0) # Change the stroke color to black
            py5.fill(204, 102, 0) # Change the fill color to light gray
            py5.rect(0, 0, 400, 160)
    
            py5.text_size(48)
            py5.fill(0) # Change the fill color to black
            py5.text_align(py5.CENTER, py5.CENTER)
            py5.text("Gratulálunk!", 0, 0)
        elif not kesz:
            py5.stroke(0) # Change the stroke color to black
            py5.fill(204, 102, 0) # Change the fill color to light gray
            py5.rect(-hossz//2+120, -szel//2+50, 200, 80)
    
            py5.text_size(24)
            py5.fill(0) # Change the fill color to black
            py5.text_align(py5.CENTER, py5.CENTER)
            py5.text("Feladod?", -hossz//2+120, -szel//2+50)
    
            py5.stroke(0)
            py5.stroke_weight(10)
            graf_csucsai.display()
            graf_elei.display()

In [17]:
#num_of_points=int(input('Mekkora gráfot szeretnél?'))
#nehezseg=int(input('Milyen nehezet szeretnél (1, 2 vagy 3)?'))
nehezseg=2
hossz=1200
szel=700
py5.run_sketch()